In [ ]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) 

import pandas as pd
import random
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import os

import numpy as np
import torch.nn.functional as F

from google.colab import drive
drive.mount('/content/drive')

from sklearn.metrics import accuracy_score
from datetime import datetime
from IPython.display import clear_output 

Mounted at /content/drive


In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.1+cu113
     |████████████████████████████████| 7.9 MB 8.3 MB/s 
     |████████████████████████████████| 2.4 MB 7.4 MB/s 
     |████████████████████████████████| 3.5 MB 8.0 MB/s 


In [ ]:
%cd /content/drive/MyDrive
label = pd.read_csv("phenotypic_training.csv")
labellist = []
for i in label["DX"]:
    if i == "ADHD":
        labellist.append([0])
    else:
        labellist.append([1])
labellist=torch.tensor(labellist)

/content/drive/MyDrive


In [ ]:
train_dataset=[]
graphlist=[]
import networkx as nx
import torch_geometric
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.utils import degree
#from torch.utils.data import DataLoader
from torch_geometric.utils import erdos_renyi_graph, to_networkx, from_networkx
%cd /content/drive/MyDrive/Training
for i in range(1, 600):
  if os.path.isfile("sub-"+ "{0:0>3}".format(i)+"/timeseries_aal.csv"):
    tmp = pd.read_csv("sub-"+ "{0:0>3}".format(i)+"/timeseries_aal.csv", header = None)
    tmp = tmp.T
    tmp = tmp.corr(method="pearson")
    tmp=(tmp.to_numpy())
    tmp=torch.from_numpy(tmp) 
    tmp=torch.div(tmp,tmp.max())
    G = nx.from_numpy_matrix(np.array(tmp))
    data = from_networkx(G)
    deg = degree(data.edge_index[0], data.num_nodes)
    x = []
    for i in deg:
      x.extend([i])
    x = np.array(x).reshape(116,1)
    data = from_networkx(G)
    #x = torch.tensor([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1] ], dtype=torch.float)
    data.x = torch.FloatTensor(x)
    #G = nx.from_numpy_matrix(np.array(tmp))
    #data = from_networkx(G)
    #x = torch.tensor([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1] ], dtype=torch.float)
    #data.x = x
    graphlist.append(data)

    #graphlist.append(tmp.type(torch.FloatTensor))

/content/drive/MyDrive/Training


In [ ]:
for i in range(len(graphlist)):
  graphlist[i].y = int(labellist[i])

In [ ]:
graphlist[0].edge_index

tensor([[  0,   0,   0,  ..., 115, 115, 115],
        [  0,   1,   2,  ..., 113, 114, 115]])

In [ ]:
dataset = random.sample(graphlist, len(graphlist))
train_dataset = dataset[:150]
test_dataset = dataset[150:]
train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=25, shuffle=True)

In [ ]:
from torch_geometric.nn import GATConv, GraphConv, GCNConv
from torch_geometric.nn.conv.gatv2_conv import GATv2Conv
from torch_geometric.nn import global_mean_pool, global_sort_pool

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()

        self.in_head = 8
        self.out_head = 1
        self.conv1 = GCNConv(in_channels=1, out_channels=16)
        self.conv2 = GCNConv(in_channels=16, out_channels=32)
        self.conv3 = GCNConv(in_channels=32, out_channels=32)
        self.lin1 = torch.nn.Linear(32, 16)
        self.lin2 = torch.nn.Linear(16, 1)
        self.sig = torch.nn.Sigmoid()

    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.weight, data.batch     
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #print(x.size())
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        #print(x.size())
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        #print(x.size())
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        #x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        #print(x)
        
        return x
        #return F.log_softmax(x, dim=1)

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 1000):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7138
Epoch: 002, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6891
Epoch: 003, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6925
Epoch: 004, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.7264
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6931
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6995
Epoch: 007, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6966
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6938
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7259
Epoch: 010, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.7041
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6915
Epoch: 012, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.7059
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6853
Epoch: 014, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6970
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6940
Epoch: 016, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6905
Epoch: 0

In [ ]:
from torch_geometric.nn import GATConv, GraphConv
from torch_geometric.nn.conv.gatv2_conv import GATv2Conv
from torch_geometric.nn import global_mean_pool, global_sort_pool

class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()

        self.in_head = 8
        self.out_head = 1
        self.conv1 = GraphConv(in_channels=1, out_channels=16)
        self.conv2 = GraphConv(in_channels=16, out_channels=32)
        self.conv3 = GraphConv(in_channels=32, out_channels=32)
        self.lin1 = torch.nn.Linear(32, 16)
        self.lin2 = torch.nn.Linear(16, 1)
        self.sig = torch.nn.Sigmoid()

    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch     
        x = self.conv1(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = self.conv2(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = self.conv3(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        #x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin1(x)
        x = self.lin2(x)
        #print(x)
        
        return x
        #return F.log_softmax(x, dim=1)

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 1000):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 5556.7080
Epoch: 002, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 15005.6396
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 362.0324
Epoch: 004, Train Acc: 0.4600, Test Acc: 0.5200, Loss: 7411.8809
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 3403.7651
Epoch: 006, Train Acc: 0.5200, Test Acc: 0.4600, Loss: 33.9692
Epoch: 007, Train Acc: 0.4733, Test Acc: 0.5200, Loss: 100.5981
Epoch: 008, Train Acc: 0.5333, Test Acc: 0.5200, Loss: 45.2705
Epoch: 009, Train Acc: 0.5267, Test Acc: 0.5200, Loss: 24.1386
Epoch: 010, Train Acc: 0.4800, Test Acc: 0.5400, Loss: 30.1621
Epoch: 011, Train Acc: 0.4867, Test Acc: 0.5200, Loss: 14.3128
Epoch: 012, Train Acc: 0.5400, Test Acc: 0.5400, Loss: 8.8665
Epoch: 013, Train Acc: 0.5800, Test Acc: 0.5200, Loss: 15.1642
Epoch: 014, Train Acc: 0.4933, Test Acc: 0.4800, Loss: 20.5059
Epoch: 015, Train Acc: 0.5333, Test Acc: 0.5400, Loss: 33.6038
Epoch: 016, Train Acc: 0.4800, Test Acc: 0.50

In [ ]:
from torch.nn import Linear
from torch_geometric.nn import GINConv, JumpingKnowledge, global_mean_pool
from torch.nn import BatchNorm1d as BN
from torch.nn import Linear, ReLU, Sequential
#from torch_geometric.nn import JumpingKnowledge, SAGEConv, global_mean_pool

class GIN(torch.nn.Module):
    def __init__(self):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            Sequential(
                Linear(1, 16),
                ReLU(),
                Linear(16, 16),
                ReLU(),
                BN(16),
            ), train_eps=False)
        self.conv2 = GINConv(
            Sequential(
                Linear(16, 32),
                ReLU(),
                Linear(32, 32),
                ReLU(),
                BN(32),
            ), train_eps=False)
        self.conv3 = GINConv(
            Sequential(
                Linear(32, 32),
                ReLU(),
                Linear(32, 32),
                ReLU(),
                BN(32),
            ), train_eps=False)
        self.lin1 = Linear(32, 16)
        self.lin2 = Linear(16, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GIN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 1000):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6937
Epoch: 002, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6936
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6987
Epoch: 004, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6971
Epoch: 005, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6928
Epoch: 006, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6934
Epoch: 007, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6935
Epoch: 008, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6970
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6951
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6934
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6933
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6965
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6929
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6903
Epoch: 015, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6910
Epoch: 016, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6978
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.data import Batch
from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
    graclus,
    max_pool,
)

num_layers  = 4
hidden = 16
class Graclus(torch.nn.Module):
    def __init__(self):
        super(Graclus, self).__init__()
        self.conv1 = GraphConv(1, 16, aggr='mean')
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(GraphConv(16, 16, aggr='mean'))
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(64, 16)
        self.lin2 = Linear(16, 1)

    def forward(self, data):
        x, edge_index, weight , batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                cluster = graclus(edge_index,num_nodes=x.size(0))
                data = Batch(x=x, edge_index=edge_index, batch=batch)
                data = max_pool(cluster, data)
                x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return x

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Graclus().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 1000):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 2.7355
Epoch: 002, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 1.7174
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.8836
Epoch: 004, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 1.0012
Epoch: 005, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6959
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6599
Epoch: 007, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.8050
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.8008
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7919
Epoch: 010, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.8245
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7160
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6866
Epoch: 013, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 1.1079
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.8318
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.8086
Epoch: 016, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.7886
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    TopKPooling,
    global_mean_pool,
)


class TopK(torch.nn.Module):
    def __init__(self):
        super(TopK, self).__init__()
        self.ratio = 0.8
        self.num_layers = 3
        self.hidden = 16
        self.conv1 = GraphConv(1, 16, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(16, 16, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [TopKPooling(16, self.ratio) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(48, 16)
        self.lin2 = Linear(16, 1)


    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, _, batch, _, _ = pool(x, edge_index,
                                                     batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        #return F.log_softmax(x, dim=-1)
        return x

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TopK().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.8179
Epoch: 002, Train Acc: 0.5267, Test Acc: 0.4800, Loss: 0.6556
Epoch: 003, Train Acc: 0.5333, Test Acc: 0.5200, Loss: 0.7094
Epoch: 004, Train Acc: 0.5800, Test Acc: 0.5200, Loss: 0.9873
Epoch: 005, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6762
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.8284
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.5200, Loss: 1.2787
Epoch: 008, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.7605
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6990
Epoch: 010, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.7228
Epoch: 011, Train Acc: 0.5400, Test Acc: 0.5200, Loss: 0.7253
Epoch: 012, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6735
Epoch: 013, Train Acc: 0.5333, Test Acc: 0.5200, Loss: 0.6720
Epoch: 014, Train Acc: 0.5800, Test Acc: 0.5600, Loss: 0.6518
Epoch: 015, Train Acc: 0.5733, Test Acc: 0.5600, Loss: 0.6871
Epoch: 016, Train Acc: 0.5800, Test Acc: 0.5400, Loss: 0.6289
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    SAGPooling,
    global_mean_pool,
)


class SAGPool(torch.nn.Module):
    def __init__(self):
        super(SAGPool, self).__init__()
        self.num_layers = 4
        self.hidden = 32
        self.ratio = 0.8
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [SAGPooling(self.hidden, self.ratio) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, _, batch, _, _ = pool(x, edge_index,
                                                     batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SAGPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 1.8831
Epoch: 002, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 1.0008
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.5724
Epoch: 004, Train Acc: 0.5267, Test Acc: 0.5600, Loss: 0.7046
Epoch: 005, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6801
Epoch: 006, Train Acc: 0.5867, Test Acc: 0.6000, Loss: 0.6911
Epoch: 007, Train Acc: 0.5467, Test Acc: 0.5200, Loss: 0.6743
Epoch: 008, Train Acc: 0.5200, Test Acc: 0.5400, Loss: 0.6949
Epoch: 009, Train Acc: 0.5800, Test Acc: 0.5000, Loss: 0.6954
Epoch: 010, Train Acc: 0.5733, Test Acc: 0.5800, Loss: 0.7067
Epoch: 011, Train Acc: 0.5867, Test Acc: 0.5000, Loss: 0.6864
Epoch: 012, Train Acc: 0.5867, Test Acc: 0.5800, Loss: 0.6788
Epoch: 013, Train Acc: 0.5467, Test Acc: 0.5400, Loss: 0.6968
Epoch: 014, Train Acc: 0.5733, Test Acc: 0.5600, Loss: 0.6698
Epoch: 015, Train Acc: 0.5733, Test Acc: 0.4800, Loss: 0.7601
Epoch: 016, Train Acc: 0.5800, Test Acc: 0.5600, Loss: 0.6938
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    EdgePooling,
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
)


class EdgePool(torch.nn.Module):
    def __init__(self):
        super(EdgePool, self).__init__()
        self.num_layers = 3 
        self.hidden = 16
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [EdgePooling(self.hidden) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, weight ,  batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, batch, _ = pool(x, edge_index, batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EdgePool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 3.5202
Epoch: 002, Train Acc: 0.5267, Test Acc: 0.5200, Loss: 1.1611
Epoch: 003, Train Acc: 0.5467, Test Acc: 0.5600, Loss: 0.8524
Epoch: 004, Train Acc: 0.5867, Test Acc: 0.5600, Loss: 0.5824
Epoch: 005, Train Acc: 0.5800, Test Acc: 0.5400, Loss: 0.7823
Epoch: 006, Train Acc: 0.5867, Test Acc: 0.5800, Loss: 0.6899
Epoch: 007, Train Acc: 0.5467, Test Acc: 0.5200, Loss: 0.8507
Epoch: 008, Train Acc: 0.5267, Test Acc: 0.5400, Loss: 0.9545
Epoch: 009, Train Acc: 0.5400, Test Acc: 0.5200, Loss: 0.6936
Epoch: 010, Train Acc: 0.5133, Test Acc: 0.5200, Loss: 0.6336
Epoch: 011, Train Acc: 0.5333, Test Acc: 0.5200, Loss: 0.6718
Epoch: 012, Train Acc: 0.5333, Test Acc: 0.5400, Loss: 0.6361
Epoch: 013, Train Acc: 0.5933, Test Acc: 0.5200, Loss: 0.7651
Epoch: 014, Train Acc: 0.5333, Test Acc: 0.5200, Loss: 0.6490
Epoch: 015, Train Acc: 0.6133, Test Acc: 0.5400, Loss: 0.6902
Epoch: 016, Train Acc: 0.5800, Test Acc: 0.6000, Loss: 0.6763
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import GlobalAttention, SAGEConv


class GlobalAttentionNet(torch.nn.Module):
    def __init__(self):
        super(GlobalAttentionNet, self).__init__()
        self.num_layers = 3
        self.hidden  = 16
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.att = GlobalAttention(Linear(self.hidden, 1))
        self.lin1 = Linear(self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.att(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)


In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GlobalAttentionNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.GlobalAttention' is deprecated, use 'nn.aggr.AttentionalAggregation' instead
  warnings.warn(out)


Epoch: 001, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6974
Epoch: 002, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6994
Epoch: 003, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.7042
Epoch: 004, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6865
Epoch: 005, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.8301
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6781
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6743
Epoch: 008, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6802
Epoch: 009, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6928
Epoch: 010, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6970
Epoch: 011, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6989
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7617
Epoch: 013, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.7175
Epoch: 014, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.7089
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7098
Epoch: 016, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6832
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import SAGEConv, Set2Set


class Set2SetNet(torch.nn.Module):
    def __init__(self):
        super(Set2SetNet,self).__init__()
        self.hidden = 16
        self.num_layers = 3 
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.set2set = Set2Set(self.hidden, processing_steps=4)
        self.lin1 = Linear(2 * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.set2set(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Set2SetNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.Set2Set' is deprecated, use 'nn.aggr.Set2Set' instead
  warnings.warn(out)


Epoch: 001, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6965
Epoch: 002, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6980
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6963
Epoch: 004, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6925
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7030
Epoch: 006, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6949
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6946
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6921
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6944
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6950
Epoch: 011, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6951
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6941
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6950
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6954
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6905
Epoch: 016, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6948
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Conv1d, Linear

from torch_geometric.nn import SAGEConv, global_sort_pool


class SortPool(torch.nn.Module):
    def __init__(self):
        super(SortPool, self).__init__()
        self.k = 30
        self.num_layers = 3 
        self.hidden = 16
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.conv1d = Conv1d(self.hidden, 32, 5)
        self.lin1 = Linear(32 * (self.k - 5 + 1), self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = global_sort_pool(x, batch, self.k)
        x = x.view(len(x), self.k, -1).permute(0, 2, 1)
        x = F.relu(self.conv1d(x))
        x = x.view(len(x), -1)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SortPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.global_sort_pool' is deprecated, use 'nn.aggr.SortAggr' instead
  warnings.warn(out)


Epoch: 001, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6924
Epoch: 002, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7593
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6971
Epoch: 004, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7002
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6957
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6902
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6976
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6927
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6938
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6927
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6909
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6974
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6916
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6960
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6937
Epoch: 016, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.6960
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    ASAPooling,
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
)


class ASAP(torch.nn.Module):
    def __init__(self):
        super(ASAP, self).__init__()
        self.num_layers = 5 
        self.hidden  = 32
        self.ratio = 0.8
        self.dropout = 0.2
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend([
            ASAPooling(self.hidden, self.ratio, dropout=self.dropout)
            for i in range((self.num_layers) // 2)
        ])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index,data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = conv(x=x, edge_index=edge_index, edge_weight=edge_weight)
            x = F.relu(x)
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, edge_weight, batch, _ = pool(
                    x=x, edge_index=edge_index, edge_weight=edge_weight,
                    batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)


In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ASAP().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 4.9804
Epoch: 002, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 1.6175
Epoch: 003, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 1.1788
Epoch: 004, Train Acc: 0.5067, Test Acc: 0.4800, Loss: 0.7295
Epoch: 005, Train Acc: 0.4933, Test Acc: 0.5200, Loss: 0.6958
Epoch: 006, Train Acc: 0.5333, Test Acc: 0.5200, Loss: 0.6778
Epoch: 007, Train Acc: 0.5933, Test Acc: 0.5600, Loss: 0.6278
Epoch: 008, Train Acc: 0.5133, Test Acc: 0.5600, Loss: 0.7863
Epoch: 009, Train Acc: 0.5400, Test Acc: 0.5200, Loss: 0.7118
Epoch: 010, Train Acc: 0.5200, Test Acc: 0.5600, Loss: 0.6965
Epoch: 011, Train Acc: 0.5133, Test Acc: 0.5200, Loss: 0.7286
Epoch: 012, Train Acc: 0.5200, Test Acc: 0.4800, Loss: 0.6729
Epoch: 013, Train Acc: 0.5267, Test Acc: 0.5400, Loss: 0.6723
Epoch: 014, Train Acc: 0.5467, Test Acc: 0.5200, Loss: 0.6780
Epoch: 015, Train Acc: 0.5867, Test Acc: 0.5400, Loss: 0.6796
Epoch: 016, Train Acc: 0.5667, Test Acc: 0.5600, Loss: 0.7375
Epoch: 0